In [18]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
import numpy as np
import joblib

import warnings
warnings.filterwarnings('ignore')




# Carga de datos

In [19]:
df=pd.read_csv("../data/ds_salaries_.csv")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


In [20]:
# Mapeo de Experience Level
experience_level_map = {
    'SE': 1,  # Junior
    'MI': 2,  # Mid-level
    'EX': 3,  # Senior
    'EN': 4   # Expert
}

# Mapeo de Company Size (Esto también podría ser OneHotEncoding si no hay un orden claro)
company_size_map = {
    'S': 1,   # Small
    'M': 2,   # Medium
    'L': 3    # Large
}

df['experience_level_encoded'] = df['experience_level'].map(experience_level_map)
df['company_size_encoded'] = df['company_size'].map(company_size_map)

In [21]:
def group_by_salary(df, column):
    # Definir los intervalos de salario
    salary_bins = [0, 60000, 90000, 150000, 200000, float('inf')]
    salary_labels = ['Low', 'Medium', 'High', 'Very High', 'Extremely High']
    
    # Crear la nueva columna con los grupos basados en salario
    df[column + '_grouped'] = pd.cut(df['salary_in_usd'], bins=salary_bins, labels=salary_labels, right=False)
    return df

# Agrupar las variables que tienen alto cardinalidad por salario
df = group_by_salary(df, 'job_title')
df = group_by_salary(df, 'employee_residence')
df = group_by_salary(df, 'company_location')
df = group_by_salary(df, 'salary_currency')

In [22]:
df=df.drop(['job_title','salary_currency', 'employee_residence','company_location'],axis=1)

In [23]:
# Seleccionar las columnas categóricas
categorical_columns = ['employment_type','experience_level_encoded','company_size_encoded', 'job_title_grouped', 'employee_residence_grouped',
                       'company_location_grouped', 'salary_currency_grouped',]

# Crear un transformer para OneHotEncoding
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Pipeline para transformar las variables categóricas
categorical_transformer = Pipeline(steps=[
    ('impute_mode', SimpleImputer(strategy='most_frequent')),
    ('encoder', encoder)
])

In [24]:
# Crear una función para el logaritmo
def log_transform(x):
    return np.log1p(x)  # Usamos log1p para evitar problemas con ceros

# Transformar salary_in_usd y otras variables numéricas
numerical_columns = ['salary_in_usd', 'salary','remote_ratio','work_year']

# Crear el pipeline para las transformaciones numéricas
numerical_transformer = Pipeline(steps=[
    ('impute', SimpleImputer()),
    ('log', FunctionTransformer(log_transform, validate=False)),
    ('scaler', MinMaxScaler())
])

# Pipelines
aqui hacemos la separacion en train y en test y despues incluimoslos pipelines que hemos creado tanto par anumericas como para categoricas en un pipeline final que es el que passamos a los modelos, importante es el x_train_transoformed, que por eso nos daba error Y SE GENERAR LAS CARPOETAS DE TRAIN Y SET TAMBIEN

In [25]:
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

train_set.to_csv("../data/salaries_train.csv")
test_set.to_csv("../data/salaries_test.csv")

X_train = train_set.copy()
y_train = train_set['salary_in_usd']
X_test = test_set.copy()
y_test = test_set['salary_in_usd']


In [26]:

# Crear el ColumnTransformer para aplicar las transformaciones a las columnas categóricas y numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ], remainder = 'drop')

# Crear el pipeline final
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Ajustar el pipeline a los datos de entrenamiento
pipeline.fit(X_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('log',
                                                                   FunctionTransformer(func=<function log_transform at 0x137052c00>)),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['salary_in_usd', 'salary',
                                                   'remote_ratio',
                                                   'work_year']),
                                                 ('cat',
                                                  Pipeline(steps=[('impute_mode',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['employment_type',
                                                   'experience_level_encoded',
                                                   'company_size_encoded',
                                                   'job_title_grouped',
                                                   'employee_residence_grouped',
                                                   'company_location_grouped',
                                                   'salary_currency_grouped'])]))])

In [27]:
X_train_transformed = pipeline.transform(X_train)
X_test_transformed = pipeline.transform(X_test)

In [28]:


# Guardar el pipeline en un archivo
joblib.dump(pipeline, "../models/pipeline_preprocesamiento.joblib")

# Para cargarlo más adelante:
# pipeline_cargado = joblib.load("pipeline_preprocesamiento.pkl")

['../models/pipeline_preprocesamiento.joblib']

# MODELOS
en primer lugar hacemos un diccionario con los modelos y sus resultados de validacion cruzda para encontrar el mejor, los mejores sales gadient boosting y  random forest, asi que empiezo con radom forest lo hago con pca sale peor y entreno el modelo para predecir, luego lo hare con el gradient boosting que solo tienen un error de 980 euros buscare sus hiperparametros lo optimizo hago predicciones y los comparo ambos 

In [29]:
# Diccionario de modelos
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'XGBoost': xgb.XGBRegressor(objective='reg:squarederror', random_state=42, verbosity=0),
    'LightGBM': lgb.LGBMRegressor(random_state=42, verbose=-1)
}

# Evaluación con validación cruzada
cv_results = {}

for name, model in models.items():
    try:
        score = np.mean(-cross_val_score(model, X_train_transformed, y_train, cv=5, scoring='neg_root_mean_squared_error'))
        cv_results[name] = score
        print(f"{name}: RMSE {score:.2f}")
    except Exception as e:
        print(f"⚠️ Error en {name}: {e}")

# Mostrar resultados
print("\nResultados de validación cruzada:", cv_results)

Linear Regression: RMSE 15649.70
Random Forest: RMSE 1089.40
Gradient Boosting: RMSE 980.71
XGBoost: RMSE 1867.00
LightGBM: RMSE 2897.70

Resultados de validación cruzada: {'Linear Regression': np.float64(15649.70334661685), 'Random Forest': np.float64(1089.3977138161324), 'Gradient Boosting': np.float64(980.7141334555469), 'XGBoost': np.float64(1867.0046630859374), 'LightGBM': np.float64(2897.7009706755775)}
